In [6]:
import time
import json
import sys
import datetime
import os
sys.path.append('PriceRequest_UberAPI/')
from itertools import permutations
#from PriceRequest_UberAPI import uberpricing
#from messageProducer import genMessage

In [7]:
from kafka import KafkaClient, KeyedProducer, SimpleProducer

'''This is a program to send message to Kafka Topic. Start and End point is used as unique Key for the Keyed producer'''
kafka=KafkaClient("localhost:9092")
producer = KeyedProducer(kafka)
def genMessage(msg,key,topic):
    producer.send_messages(topic,key, msg)

In [8]:
from uber_rides.session import Session
from uber_rides.client import UberRidesClient

session = Session(server_token='DGdtuZb4i7zwENbYaVSPwuXLBtnzwkI2DigBmtmf')
def getPriceEstimate(key,start_lat,start_long,end_lat,end_long):
    client = UberRidesClient(session)
    p=client.get_price_estimates(start_lat,start_long,end_lat,end_long)
    k_lat_long=key+"|"+str(start_lat)+"|"+str(start_long)+"|"+str(end_lat)+"|"+str(end_long)
    value=p.json.get('prices')
    #value['key'] = k_lat_long
    return (value),str(k_lat_long)

In [9]:
#List of Locations
loc1=(37.787817,-122.396595)#Galvanize
loc2=(37.776908,-122.394825)#Caltrain
loc3=(37.810470, -122.476624)#
loc4=(37.775505, -122.446444)#USF
loc5=(37.810604,-122.409856)#Pier39
loc6=(37.687802, -122.470648)#Daly City
loc7=(37.737842, -122.431546)#GlenPark
loc8=(37.768719, -122.488666)#GoldenGatePark
loc9=(37.621206, -122.388759)#Airport
loc10=(37.692554, -122.432361)#San Bruno MOuntain State Park

location_dict={}
permutation_dict={}
location_dict.update({'A':loc1,
                      'B':loc2,
                      'C':loc3,
                      'D':loc4,
                      'E':loc5,
                      'F':loc6, 
                      'G':loc7, 
                      'H':loc8,
                      'G':loc9,
                      'I':loc10})

loc_list=[loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10]
topic="test"

#total_route=list(permutations(location_dict.itervalues(),2))
#ordered_route=sorted(total_route,key=(lambda x:x[1]))

total_route=list(permutations(location_dict,2))
ordered_route=sorted(total_route,key=(lambda x:x[0]))
for start,dest in ordered_route:
    key = start+'-'+dest
    permutation_dict[key] = [(location_dict[start],location_dict[dest])]

In [11]:
'''This is a job scheduler which sends a request to Uber API 
to get p
ricing Details and then send message to Kafka Topic.
'''

for route in permutation_dict:
    message,key=getPriceEstimate(route,permutation_dict[route][0][0][0],permutation_dict[route][0][0][1],permutation_dict[route][0][1][0],permutation_dict[route][0][1][1])
    print message,key
    #genMessage(message,key,topic)
    

[{u'localized_display_name': u'SELECT', u'distance': 4.19, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 29.0, u'low_estimate': 23.0, u'duration': 1020, u'estimate': u'$23-29', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 4.19, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 20.0, u'low_estimate': 16.0, u'duration': 1020, u'estimate': u'$16-20', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 4.19, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 41.0, u'low_estimate': 32.0, u'duration': 1020, u'estimate': u'$32-41', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 4.19, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 53.0, u'low_estimate': 42.0, u'duration': 1020, u'estimate': u'$42-53'

[{u'localized_display_name': u'SELECT', u'distance': 12.42, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 66.0, u'low_estimate': 53.0, u'duration': 1380, u'estimate': u'$53-66', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 12.42, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 47.0, u'low_estimate': 37.0, u'duration': 1380, u'estimate': u'$37-47', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 12.42, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 78.0, u'low_estimate': 63.0, u'duration': 1380, u'estimate': u'$63-78', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 12.42, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 92.0, u'low_estimate': 74.0, u'duration': 1380, u'estimate': u'$74

[{u'localized_display_name': u'SELECT', u'distance': 3.94, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 29.0, u'low_estimate': 23.0, u'duration': 600, u'estimate': u'$23-29', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 3.94, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 21.0, u'low_estimate': 16.0, u'duration': 600, u'estimate': u'$16-21', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 3.94, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 34.0, u'low_estimate': 27.0, u'duration': 600, u'estimate': u'$27-34', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 3.94, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 44.0, u'low_estimate': 35.0, u'duration': 600, u'estimate': u'$35-44', u'

[{u'localized_display_name': u'SELECT', u'distance': 13.27, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 65.0, u'low_estimate': 53.0, u'duration': 1680, u'estimate': u'$53-65', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 13.27, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 46.0, u'low_estimate': 37.0, u'duration': 1680, u'estimate': u'$37-46', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 13.27, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 86.0, u'low_estimate': 69.0, u'duration': 1680, u'estimate': u'$69-86', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 13.27, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 100.0, u'low_estimate': 81.0, u'duration': 1680, u'estimate': u'$8

[{u'localized_display_name': u'SELECT', u'distance': 5.71, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 39.0, u'low_estimate': 31.0, u'duration': 1020, u'estimate': u'$31-39', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 5.71, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 27.0, u'low_estimate': 21.0, u'duration': 1020, u'estimate': u'$21-27', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 5.71, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 46.0, u'low_estimate': 37.0, u'duration': 1020, u'estimate': u'$37-46', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 5.71, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 58.0, u'low_estimate': 46.0, u'duration': 1020, u'estimate': u'$46-58'

...

[{u'localized_display_name': u'SELECT', u'distance': 7.84, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 45.0, u'low_estimate': 36.0, u'duration': 1020, u'estimate': u'$36-45', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 7.84, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 32.0, u'low_estimate': 25.0, u'duration': 1020, u'estimate': u'$25-32', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 7.84, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 58.0, u'low_estimate': 47.0, u'duration': 1020, u'estimate': u'$47-58', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 7.84, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 71.0, u'low_estimate': 57.0, u'duration': 1020, u'estimate': u'$57-71'

[{u'localized_display_name': u'SELECT', u'distance': 12.67, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 57.0, u'low_estimate': 46.0, u'duration': 1020, u'estimate': u'$46-57', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 12.67, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 42.0, u'low_estimate': 33.0, u'duration': 1020, u'estimate': u'$33-42', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 12.67, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 77.0, u'low_estimate': 62.0, u'duration': 1020, u'estimate': u'$62-77', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 12.67, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 94.0, u'low_estimate': 76.0, u'duration': 1020, u'estimate': u'$76

[{u'localized_display_name': u'SELECT', u'distance': 15.36, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 68.0, u'low_estimate': 55.0, u'duration': 1500, u'estimate': u'$55-68', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 15.36, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 50.0, u'low_estimate': 40.0, u'duration': 1500, u'estimate': u'$40-50', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 15.36, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 77.0, u'low_estimate': 62.0, u'duration': 1500, u'estimate': u'$62-77', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 15.36, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 94.0, u'low_estimate': 76.0, u'duration': 1500, u'estimate': u'$76

[{u'localized_display_name': u'SELECT', u'distance': 7.4, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 39.0, u'low_estimate': 31.0, u'duration': 840, u'estimate': u'$31-39', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 7.4, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 28.0, u'low_estimate': 22.0, u'duration': 840, u'estimate': u'$22-28', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 7.4, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 51.0, u'low_estimate': 40.0, u'duration': 840, u'estimate': u'$40-51', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 7.4, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 61.0, u'low_estimate': 49.0, u'duration': 840, u'estimate': u'$49-61', u'curr

[{u'localized_display_name': u'SELECT', u'distance': 11.92, u'display_name': u'SELECT', u'product_id': u'57c0ff4e-1493-4ef9-a4df-6b961525cf92', u'high_estimate': 50.0, u'low_estimate': 40.0, u'duration': 1080, u'estimate': u'$40-50', u'currency_code': u'USD'}, {u'localized_display_name': u'uberXL', u'distance': 11.92, u'display_name': u'uberXL', u'product_id': u'821415d8-3bd5-4e27-9604-194e4359a449', u'high_estimate': 36.0, u'low_estimate': 29.0, u'duration': 1080, u'estimate': u'$29-36', u'currency_code': u'USD'}, {u'localized_display_name': u'BLACK', u'distance': 11.92, u'display_name': u'BLACK', u'product_id': u'd4abaae7-f4d6-4152-91cc-77523e8165a4', u'high_estimate': 73.0, u'low_estimate': 59.0, u'duration': 1080, u'estimate': u'$59-73', u'currency_code': u'USD'}, {u'localized_display_name': u'SUV', u'distance': 11.92, u'display_name': u'SUV', u'product_id': u'8920cb5e-51a4-4fa4-acdf-dd86c5e18ae0', u'high_estimate': 85.0, u'low_estimate': 68.0, u'duration': 1080, u'estimate': u'$68

In [64]:
import ast
from google.cloud import bigtable
from google.cloud import happybase
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\Users\Pradeep\Google Drive\Uber_Pricing_Pipelling\UBER-Price-Surge-fe496f50d763.json"
#client_credentials = json.load(open("<path to .json credentials>"))
project_id = 'uber-price-surge'
instance_id = 'uber-price' 
table_name = 'uber_price'
cluster = 'uber-price-cluster'
zone = 'us-east1-b'

def connection(project_id, instance_id ):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    connection = happybase.Connection(instance=instance)
    return connection

connection = connection(project_id, instance_id )
table = connection.table(table_name)

In [25]:
print('Inserting values into the {} table.'.format(table_name))

table.put(value[0], {column_name:json.dumps(value[1])})

Inserting values into the price_table table.


NameError: name 'value' is not defined

In [68]:
for route in permutation_dict:
    message,key=getPriceEstimate(route,permutation_dict[route][0][0][0],permutation_dict[route][0][0][1],permutation_dict[route][0][1][0],permutation_dict[route][0][1][1])
    
    for i in message:

        #val_dict = ast.literal_eval(i)
        key = key+ str(datetime.datetime.now()) 
        val_dict = i
        if (val_dict['low_estimate'] != None and  val_dict['high_estimate']!= None):
            dic_values={'distance': val_dict['distance'],
                    'low_estimate':val_dict['low_estimate'],
                    'high_estimate': val_dict['high_estimate'],
                    'price' : (val_dict['low_estimate']+val_dict['high_estimate'])/2,
                    'uber_type': val_dict['display_name']             
                   }
            column_family_name= dic_values['uber_type']
            column_name = '{fam}:price'.format(fam=column_family_name)
            table.put(key, {column_name:json.dumps(val_dict)})
            
 
    